In [59]:
import os
import pandas as pd
import numpy as np
import json
from utils import nlp

In [37]:
special_values = ['none', 'yes', 'no', 'dontcare', '?']

In [69]:
feats_lstm = json.load(open('../../sc-lstm/resource/mwoz/feat.json'))
data_lstm = json.load(open('../../sc-lstm/resource/mwoz/text.json'))
split_list = json.load(open('../../sc-lstm/resource/mwoz/data_split/allDataSplitRand0925.json'))

In [22]:
test_list = [elem[0] for elem in split_list['test']]
test_list = list(set(test_list))

In [25]:
feats = {}
for elem in test_list:
    feats[elem] = feats_lstm[elem]

In [26]:
feats

{'PMUL4504': {'9': {'Train-Inform': [['Arrive', '1', '17:58'],
    ['Id', '1', 'TR3677 ']]},
  '7': {'Booking-Book': [['Ref', '1', 'FC2BDIC9']]},
  '1': {'Hotel-Request': [['Area', '?', '?']],
   'Hotel-Inform': [['Choice', '1', '23']]},
  '5': {'Hotel-NoOffer': [['Type', '1', 'hotels '],
    ['Area', '1', 'North'],
    ['Price', '1', 'expensive ']],
   'general-reqmore': [['none', 'none', 'none']]},
  '8': {'Train-Request': [['Leave', '?', '?'], ['Arrive', '?', '?']]},
  '10': {'general-bye': [['none', 'none', 'none']],
   'general-welcome': [['none', 'none', 'none']]},
  '4': {'Hotel-Request': [['Parking', '?', '?']]},
  '3': {'Hotel-Inform': [['Area', '1', 'north'],
    ['Price', '1', 'moderately priced'],
    ['Choice', '1', '8'],
    ['Type', '1', 'guesthouses ']],
   'Hotel-Recommend': [['Name', '1', 'Acorn Guest house'],
    ['Stars', '1', '4']]},
  '6': {'Booking-Inform': [['none', 'none', 'none']],
   'Hotel-Recommend': [['Price', '1', 'moderately-priced'],
    ['Internet', 'y

In [62]:
info['PMUL4504']['9']


{'delex': 'ok the slot-train-inform-id will get you there at slot-train-inform-arrive',
 'ori': 'ok the TR3677 will get you there at 17:58'}

In [49]:
new_feat = {}
for key, dialog in feats.items():
    new_feats_dac = {}
    for k1, ft in dialog.items():
        lst_feat_diag = {}
        for k, v in ft.items():
            for val in v:
                new_key = k.lower()+'-'+val[0].lower()+'-'+val[1]
                lst_feat_diag[new_key] = val[2]
        new_feats_dac[k1] = lst_feat_diag
    new_feat[key] = new_feats_dac

In [50]:
new_feat

{'PMUL4504': {'9': {'train-inform-arrive-1': '17:58',
   'train-inform-id-1': 'TR3677 '},
  '7': {'booking-book-ref-1': 'FC2BDIC9'},
  '1': {'hotel-request-area-?': '?', 'hotel-inform-choice-1': '23'},
  '5': {'hotel-nooffer-type-1': 'hotels ',
   'hotel-nooffer-area-1': 'North',
   'hotel-nooffer-price-1': 'expensive ',
   'general-reqmore-none-none': 'none'},
  '8': {'train-request-leave-?': '?', 'train-request-arrive-?': '?'},
  '10': {'general-bye-none-none': 'none', 'general-welcome-none-none': 'none'},
  '4': {'hotel-request-parking-?': '?'},
  '3': {'hotel-inform-area-1': 'north',
   'hotel-inform-price-1': 'moderately priced',
   'hotel-inform-choice-1': '8',
   'hotel-inform-type-1': 'guesthouses ',
   'hotel-recommend-name-1': 'Acorn Guest house',
   'hotel-recommend-stars-1': '4'},
  '6': {'booking-inform-none-none': 'none',
   'hotel-recommend-price-1': 'moderately-priced',
   'hotel-recommend-internet-yes': 'none',
   'hotel-recommend-area-1': 'north',
   'hotel-recommend-

In [30]:
feats['PMUL4504']

{'9': {'Train-Inform': [['Arrive', '1', '17:58'], ['Id', '1', 'TR3677 ']]},
 '7': {'Booking-Book': [['Ref', '1', 'FC2BDIC9']]},
 '1': {'Hotel-Request': [['Area', '?', '?']],
  'Hotel-Inform': [['Choice', '1', '23']]},
 '5': {'Hotel-NoOffer': [['Type', '1', 'hotels '],
   ['Area', '1', 'North'],
   ['Price', '1', 'expensive ']],
  'general-reqmore': [['none', 'none', 'none']]},
 '8': {'Train-Request': [['Leave', '?', '?'], ['Arrive', '?', '?']]},
 '10': {'general-bye': [['none', 'none', 'none']],
  'general-welcome': [['none', 'none', 'none']]},
 '4': {'Hotel-Request': [['Parking', '?', '?']]},
 '3': {'Hotel-Inform': [['Area', '1', 'north'],
   ['Price', '1', 'moderately priced'],
   ['Choice', '1', '8'],
   ['Type', '1', 'guesthouses ']],
  'Hotel-Recommend': [['Name', '1', 'Acorn Guest house'],
   ['Stars', '1', '4']]},
 '6': {'Booking-Inform': [['none', 'none', 'none']],
  'Hotel-Recommend': [['Price', '1', 'moderately-priced'],
   ['Internet', 'yes', 'none'],
   ['Area', '1', 'north

In [65]:
lst = []
for k1, v1 in info.items():
    for k2, v2 in v1.items():
        try:
            lst.append((k1, k2, nlp.normalize(v2['delex'])))
        except:
            pass

In [66]:
lst

[('PMUL4504',
  '9',
  'ok the slot-train-inform-id will get you there at slot-train-inform-arrive'),
 ('PMUL4504',
  '7',
  'booking was successful . reference number is : slot-booking-book-ref .'),
 ('PMUL4504',
  '1',
  'we have slot-hotel-inform-choice different options available . is there an area you would prefer ?'),
 ('PMUL4504',
  '5',
  'i m sorry . i am not finding any slot-hotel-nooffer-type in the slot-hotel-nooffer-price price range in the slot-hotel-nooffer-area . would you like to change your search ?'),
 ('PMUL4504', '8', 'when would you like to leave or arrive in cambridge by ?'),
 ('PMUL4504', '10', 'you re welcome , have a great day .'),
 ('PMUL4504', '4', 'do you have a parking preference ?'),
 ('PMUL4504',
  '3',
  'there are slot-hotel-inform-choice slot-hotel-inform-price slot-hotel-inform-type in the slot-hotel-inform-area . may i suggest the slot-hotel-recommend-stars star slot-hotel-recommend-name ?'),
 ('PMUL4504',
  '6',
  'slot-hotel-recommend-name is slot

# Results sc-lstm

In [75]:
fin = open('../../sc-lstm/sclstm.res')
results_lstm = fin.readlines()

In [82]:
targets = [elem.split('Target:')[-1].strip() for elem in results_lstm if elem.startswith('Target')]
gens = [elem.split(':')[-1].strip() for elem in results_lstm if elem.startswith('Gen0')]
tg_lst = []
for idx, elem in enumerate(targets):
    tg_lst.append((elem, gens[idx]))

In [92]:
tg_lst[0]

('you do have slot-hotel-inform-choice options , including the slot-hotel-inform-name which offers free wifi as well as free parking . it is also slot-hotel-inform-price , and rated slot-hotel-inform-stars stars .',
 'there are slot-hotel-inform-choice slot-hotel-inform-stars star hotel in the slot-hotel-inform-area . slot-hotel-inform-name is slot-hotel-inform-price and has free wifi and parking .')

In [86]:
lst[0]

('PMUL4504',
 '9',
 'ok the slot-train-inform-id will get you there at slot-train-inform-arrive')

In [98]:
last_lst = []
for elem in tg_lst:
    for utt in lst:
        if nlp.normalize(elem[0]) == nlp.normalize(utt[-1]):
            last_lst.append((utt[0], utt[1], elem[1]))

In [100]:
len(last_lst)

208